In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import PIL
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, ReLU, Activation
from tensorflow.keras.layers import UpSampling2D, Conv2D, Concatenate, Dense, concatenate
from tensorflow.keras.layers import Flatten, Lambda, Reshape, ZeroPadding2D, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from tensorflow.keras.utils import to_categorical
import keras.backend as K

In [2]:
# to get access to google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# for google drive case in colab

path = "/content/drive/MyDrive/gan"

# for dataset with girls 

In [4]:
# one_hot encoding for text 
import pickle


text_path = path+"/girls_dataset/Text/00_Female/"


dataset_train_path = path+"/girls_dataset/Train/filenames.pickle"
dataset_test_path =  path+"/girls_dataset/Test/filenames.pickle"

# girls dataset open .pickle filenames

train_names = []
test_names = []

with open(dataset_train_path, 'rb') as f:
    images_train_names = pickle.load(f)
    
for name in images_train_names:
    # print(name)
    image_name = name[10:]
    # image_name = name.removeprefix("00_Female/")
    train_names.append(image_name)
    
with open(dataset_test_path, 'rb') as f:
    images_test_names = pickle.load(f)
	
for name in images_test_names:
    image_name = name[10:]
    # image_name = name.removeprefix("00_Female/")
    test_names.append(image_name)	


train_text_list = []
test_text_list = []

for file in os.listdir(text_path):
    with open(text_path+file) as f:
        a = f.read()
        # file.removesuffix(".txt")
        if file[:5] in train_names:
            train_text_list.append(a)
        if file[:5] in test_names:
            test_text_list.append(a) 
        
        

# Training Custom Word2Vec Model in Gensim

In [7]:
from gensim.models import KeyedVectors,Word2Vec
from gensim.utils import simple_preprocess

# For visualization of word2vec model
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


# Text Preprocessing
preprocessed_text_train = list(map(simple_preprocess,train_text_list))
preprocessed_text_test = list(map(simple_preprocess,test_text_list))

In [ ]:
# count the mean len of tokenized texts
mean1 = 0
min1 = 999
for token_text in preprocessed_text_train:
    mean1+=len(token_text)
    min1 = min(min1, len(token_text))
print(mean1//500)
print(min1)

mean1 = 0
min1 = 999
for token_text in preprocessed_text_test:
    mean1+=len(token_text)
    min1 = min(min1, len(token_text))
print(mean1//260)
print(min1)

169
120
171
132


Text preprocessing

In [ ]:
# training the embedding

model_cbow_train = Word2Vec(sentences=preprocessed_text_train,
sg=0, min_count=1, workers=4, window =5, iter = 100,size=9)

model_cbow_test = Word2Vec(sentences=preprocessed_text_test,
sg=0, min_count=1, workers=4, window =5, iter = 100,size=9)

In [ ]:
# make embeddedings

train_text_embedded = []
test_text_embedded = []

for index,token_sentence in enumerate(preprocessed_text_train):
    train_text_embedded.append(list())
    for word in token_sentence[:120]:
    
        try:
            vector = model_cbow_train.wv.get_vector(word)
            train_text_embedded[index].append(vector)
        except:
            print("no word as a key")
            
for index,token_sentence in enumerate(preprocessed_text_test):
    test_text_embedded.append(list())
    for word in token_sentence[:120]:
    
        try:
            vector = model_cbow_test.wv.get_vector(word)
            test_text_embedded[index].append(vector)
        except:
            print("no word as a key")

In [ ]:
# convert to mu,py array
train_text_embedded = np.array(train_text_embedded)
print(train_text_embedded.shape)
test_text_embedded = np.array(test_text_embedded)
print(test_text_embedded.shape)

(500, 120, 9)
(260, 120, 9)


In [ ]:
# save the models

import pickle

path = path+"/init_embedding_values_girls/"

with open(f'{path}embed_train.pickle', 'wb') as f:
    pickle.dump(train_text_embedded, f)

with open(f'{path}embed_test.pickle', 'wb') as f:
    pickle.dump(test_text_embedded, f)

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    
    for word in model.wv.key_to_index:
        
        tokens.append(model.wv[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model_cbow_train)

# ELMo embedding

In [8]:
import tensorflow_hub as hub
import tensorflow as tf

In [9]:
embed = hub.load("https://tfhub.dev/google/elmo/3")

In [10]:
train_text_list = []
test_text_list = []


for text in preprocessed_text_train:
  train_text_list.append(" ".join(text))

for text in preprocessed_text_test:
  test_text_list.append(" ".join(text))


In [14]:
[test_text_list[0]]

['this woman turns her face towards the front side this woman is about to years old and has big brown eyes with double eyelids and long eyelashes thick flat eyebrows and dark brown skin woman with thick bushy eyebrows big eyes with long eyelashes and square face this woman has medium nose medium glamour mouth and square shape face this woman is about to years old with brown braid hair long shape face and olive skin this woman has big bright eyes thick bushy eyebrows medium nose and natural mouth woman has long shape face dark brown skin and brown braid hair woman has brown hair with pink hood covering ears and olive skin the woman has square shape face with big nose natural mouth big bright eyes thick flat eyebrows and long brown braid hair with pink scarf']

In [ ]:
train_text_list_embed = []
test_text_list_embed = []

for index,data_part in enumerate([train_text_list,test_text_list]):
  for text in data_part:
    if index == 0:
      train_bededding = embed.signatures['default'](tf.constant([text]))
      train_text_list_embed.append(train_bededding["default"])
    else:
      test_bededding = embed.signatures['default'](tf.constant([text]))
      test_text_list_embed.append(test_bededding["default"])

In [ ]:
train_bededding = np.array(train_text_list_embed)
test_bededding = np.array(test_text_list_embed)

In [ ]:
train_bededding.shape

(1, 1024)